# SODA Toolchain

![tutorial-flow](imgs/flow-diagram.png)

# High-Level Application Input (TensorFlow)

### Build a model in TensorFlow (Step 1)

In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np
tf.random.set_seed(seed=0)
print(tf.__version__)

in1 = keras.layers.Input(shape=(32,32,1))
tmp = keras.layers.Conv2D(filters=1, kernel_size=(5,5),
                          input_shape=(32,32),
                          padding='same', 
                          strides=(2, 2),
                          activation='relu', 
                          use_bias=True)(in1)
tmp = keras.layers.Flatten()(tmp)
tmp = keras.layers.Dense(units=8, activation='relu')(tmp)
tmp = keras.layers.Dense(units=4, activation='relu')(tmp)
out = keras.layers.Dense(units=2, activation='softmax')(tmp)

model = keras.models.Model(inputs=in1, outputs=out)

# Compile model with optimizer
model.compile(optimizer="adam",
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])

2.17.1


### Convert model to protobuf

In [3]:
!mkdir -p output

In [5]:
save_path = os.path.join(os.getcwd(), "model/simple/")

# Save model to SavedModel format
tf.saved_model.save(model, save_path)

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    x=[
        tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype, name='x1')
    ])

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)

# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                    logdir=os.getcwd(),
                    name="output/frozen_graph.pbtxt",
                    as_text=True)


INFO:tensorflow:Assets written to: /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/model/simple/assets


INFO:tensorflow:Assets written to: /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/model/simple/assets
/usr/local/lib/python3.11/site-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_6
Received: inputs=['Tensor(shape=(None, 32, 32, 1))']
  warnings.warn(msg)
I0000 00:00:1748615414.208788   15580 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2025-05-30 14:30:14.208980: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


'/workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/frozen_graph.pbtxt'

![tutorial-flow](imgs/flow-diagram.png)

### Transform protobuf into MLIR (Step 2)




In [6]:
!scripts/protobuf-to-tosa.sh output/frozen_graph.pbtxt output/tosa.mlir

2025-05-30 14:30:20.617175: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Lower MLIR to Linalg on Buffers (Step 3)

In [7]:
!scripts/tosa-to-linalg.sh output/tosa.mlir output/linalg-buffers.mlir

![tutorial-flow](imgs/flow-diagram.png)

# SODA-OPT: HW/SW Partitioning and Optimizer (Step 4)

## How to use soda.launch?

### Automatic selection of custom accelerator region

Using the pass: `-convert-<abstraction_name>-<operation_name>-to-soda`

Such as: `-convert-linalg-generic-to-soda`

### Manual selection of custom accelerator region

Adding the following lines around any code that will become the accelerator:

```mlir
soda.launch {
  // ...
  // Code to be transformed into an accelerator
  // ...
  soda.terminator
}
```

Run next cell and edit [file](output/01searched-edited.mlir).

In [8]:
!cp output/linalg-buffers.mlir output/01searched-edited.mlir

# Perform manual edit!

> **⚠️ <span style="color:red;">IMPORTANT:</span> Please modify the file as described below.**

Edit the [file](output/01searched-edited.mlir).

Modify line 55 to the following lines:

```mlir
    soda.launch {
      linalg.batch_matmul ins(%expand_shape_8, %1 : memref<1x4x8xf32>, memref<1x8x4xf32>) outs(%alloc_9 : memref<1x4x4xf32>)
      soda.terminator
    }
```

## Optimization pipeline

![optimizations](imgs/optimization-table.png)

### Kernel without SODA-OPT optimizations (Baseline)

In [9]:
%%bash
(
  CURRENT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
  source $CURRENT_DIR/scripts/check_docker.sh

  $DOCKER_RUN \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-c-testbench=using-bare-ptr \
    -soda-generate-bambu-accelcode=no-aa \
    -lower-all-to-llvm=use-bare-ptr-memref-call-conv \
    -mlir-print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04baseline.mlir \
    2>&1 | cat > output/05intermediate-baseline.mlir

  $DOCKER_RUN \
  mlir-translate \
    --mlir-to-llvmir \
    output/04baseline.mlir \
    -o output/05baseline.ll
)

Visualize [intermediate file](output/05intermediate-baseline.mlir)

### Kernel with SODA-OPT optimizations

In [10]:
%%bash
(
    CURRENT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
source $CURRENT_DIR/scripts/check_docker.sh

  $DOCKER_RUN \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-c-testbench=using-bare-ptr \
    -soda-generate-bambu-accelcode \
    -soda-opt-pipeline-for-bambu=use-bare-ptr-memref-call-conv \
    -mlir-print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04optimized.mlir \
    2>&1 | cat > output/05intermediate-optimized.mlir

  $DOCKER_RUN \
  mlir-translate \
    --mlir-to-llvmir \
    output/04optimized.mlir \
    -o output/05optimized.ll
)


Visualize [intermediate file](output/05intermediate-optimized.mlir)

![tutorial-flow](imgs/flow-diagram.png)

# Bambu: Synthesizing the Outlined Kernel (Step 5)

The following configurations are passed to our backend HLS tool:

* Target: ASIC generation using the Nangate cell library with the FreePDK 45nm kit
* Memory technology: SRAM
* Number of memory channels: 2
  * Supports 2 parallel reads and 2 parallel writes
* Target frequency: 200MHz (5ns period)
* Using bambu's floating-point operation support

You can change parameters passed to bambu in [scripts/run-bambu.sh](scripts/run-bambu.sh)

### Baseline kernel

In [11]:
! scripts/run-bambu.sh baseline

/workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/baseline /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed
 ==  Bambu executed with: bambu -v3 --print-dot -lm --soft-float --compiler=I386_CLANG16 --device=nangate45 --clock-period=5 --experimental-setup=BAMBU-BALANCED-MP --channels-number=2 --memory-allocation-policy=ALL_BRAM --disable-function-proxy --generate-tb=main_kernel_testbench.c --simulate --simulator=VERILATOR --verilator-parallel --top-fname=main_kernel input.ll 


********************************************************************************
                    ____                  _
                   | __ )  __ _ _ __ ___ | |_   _   _
                   |  _ \ / _` | '_ ` _ \| '_ \| | | |
                   | |_) | (_| | | | | | | |_) | |_| |
                   |____/ \__,_|_| |_| |_|_.__/ \__,_|

********************************************************************************
                         High-Lev

In [12]:
baseline_runtime = ""

for runtime in open('output/baseline/bambu-log').readlines():
    if "Average execution" in runtime:
        baseline_runtime = [int(s) for s in runtime.split() if s.isdigit()][0]

print("Average execution in cycles: {}".format(baseline_runtime))

Average execution in cycles: 1286


Visualize [Intermediate Dot File](output/baseline/HLS_output/dot/main_kernel/HLS_STGraph.dot)

### Optimized kernel

In [13]:
! scripts/run-bambu.sh optimized
# Takes aprox 30 seconds to execute

/workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/optimized /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed
 ==  Bambu executed with: bambu -v3 --print-dot -lm --soft-float --compiler=I386_CLANG16 --device=nangate45 --clock-period=5 --experimental-setup=BAMBU-BALANCED-MP --channels-number=2 --memory-allocation-policy=ALL_BRAM --disable-function-proxy --generate-tb=main_kernel_testbench.c --simulate --simulator=VERILATOR --verilator-parallel --top-fname=main_kernel input.ll 


********************************************************************************
                    ____                  _
                   | __ )  __ _ _ __ ___ | |_   _   _
                   |  _ \ / _` | '_ ` _ \| '_ \| | | |
                   | |_) | (_| | | | | | | |_) | |_| |
                   |____/ \__,_|_| |_| |_|_.__/ \__,_|

********************************************************************************
                         High-Le

In [14]:
optimized_runtime = ""

for runtime in open('output/optimized/bambu-log').readlines():
    if "Average execution" in runtime:
        optimized_runtime = [int(s) for s in runtime.split() if s.isdigit()][0]

print("Average execution in cycles: {}".format(optimized_runtime))


Average execution in cycles: 56


Visualize [Intermediate Dot File](output/optimized/HLS_output/dot/main_kernel/HLS_STGraph.dot)

## Comparison of runtime results

* Display runtime
* Display [verilog output file](output/optimized/main_kernel.v)

In [15]:
print("Average execution in cycles of Baseline kernel:  {}".format(baseline_runtime))
print("Average execution in cycles of Optimized kernel: {}".format(optimized_runtime))
print("Speedup: {:.1f}".format(float(baseline_runtime/optimized_runtime)))

Average execution in cycles of Baseline kernel:  1286
Average execution in cycles of Optimized kernel: 56
Speedup: 23.0


# Commandline interface

To visualize all possible paramenters for our optimization passes run:

- `soda-opt -h`

```
      --soda-opt-pipeline-for-bambu                    
        --affine-tile-size=<ulong>                     
        --bitwidth-of-index-type=<uint>                
        --max-alloc-size-in-bytes=<uint>               
        --max-rank-of-allocated-memref=<uint>          
        --number-of-full-unrolls=<uint>                
        --permutation-map=<uint>                       
        --use-bare-ptr-memref-call-conv                
        --no-alloca-promotion                          
        --no-buffer-trick                              
        --no-scalar-replacement                        
  
```

In [16]:
%%bash
(
  CURRENT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
  source $CURRENT_DIR/scripts/check_docker.sh

  $DOCKER_RUN \
  soda-opt -h 2>&1 | cat > output/helpfile
)

Open [help file](output/helpfile)

### Modifying the number of unrolls

In [17]:
%%bash
(
  CURRENT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
  source $CURRENT_DIR/scripts/check_docker.sh

  $DOCKER_RUN \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-c-testbench=using-bare-ptr \
    -soda-generate-bambu-accelcode \
    -soda-opt-pipeline-for-bambu="use-bare-ptr-memref-call-conv number-of-full-unrolls=1" \
    -mlir-print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04optimized.mlir \
    2>&1 | cat > output/05intermediate-optimized.mlir

  CURRENT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
  source $CURRENT_DIR/scripts/check_docker.sh
  $DOCKER_RUN \
  mlir-translate \
    --mlir-to-llvmir \
    output/04optimized.mlir \
    -o output/05optimized.ll
)

Visualize [intermediate file](output/05intermediate-optimized.mlir)

In [18]:
! scripts/run-bambu.sh optimized

/workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/optimized /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed
 ==  Bambu executed with: bambu -v3 --print-dot -lm --soft-float --compiler=I386_CLANG16 --device=nangate45 --clock-period=5 --experimental-setup=BAMBU-BALANCED-MP --channels-number=2 --memory-allocation-policy=ALL_BRAM --disable-function-proxy --generate-tb=main_kernel_testbench.c --simulate --simulator=VERILATOR --verilator-parallel --top-fname=main_kernel input.ll 


********************************************************************************
                    ____                  _
                   | __ )  __ _ _ __ ___ | |_   _   _
                   |  _ \ / _` | '_ ` _ \| '_ \| | | |
                   | |_) | (_| | | | | | | |_) | |_| |
                   |____/ \__,_|_| |_| |_|_.__/ \__,_|

********************************************************************************
                         High-Le

### Default optimization pipeline (again)

Three full unrolls of the inner loop yield better latency for this kernel.

In [19]:
%%bash
(
  CURRENT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
  source $CURRENT_DIR/scripts/check_docker.sh

  $DOCKER_RUN \
  soda-opt \
    -soda-outline-bambu-code \
    -soda-extract-arguments-to-c-testbench=using-bare-ptr \
    -soda-generate-bambu-accelcode \
    -soda-opt-pipeline-for-bambu="use-bare-ptr-memref-call-conv" \
    -mlir-print-ir-after-all \
    output/01searched-edited.mlir \
    -o output/04optimized.mlir \
    2>&1 | cat > output/05intermediate-optimized.mlir

  CURRENT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
  source $CURRENT_DIR/scripts/check_docker.sh
  $DOCKER_RUN \
  mlir-translate \
    --mlir-to-llvmir \
    output/04optimized.mlir \
    -o output/05optimized.ll
)

In [20]:
! scripts/run-bambu.sh optimized

/workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/optimized /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed
 ==  Bambu executed with: bambu -v3 --print-dot -lm --soft-float --compiler=I386_CLANG16 --device=nangate45 --clock-period=5 --experimental-setup=BAMBU-BALANCED-MP --channels-number=2 --memory-allocation-policy=ALL_BRAM --disable-function-proxy --generate-tb=main_kernel_testbench.c --simulate --simulator=VERILATOR --verilator-parallel --top-fname=main_kernel input.ll 


********************************************************************************
                    ____                  _
                   | __ )  __ _ _ __ ___ | |_   _   _
                   |  _ \ / _` | '_ ` _ \| '_ \| | | |
                   | |_) | (_| | | | | | | |_) | |_| |
                   |____/ \__,_|_| |_| |_|_.__/ \__,_|

********************************************************************************
                         High-Le

![tutorial-flow](imgs/flow-diagram.png)

# OpenRoad Flow: Automatic ASIC place and route (Step 6)

### Baseline kernel

In [21]:
! scripts/run-openroad.sh baseline

# Approx. 5min to execute

/workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/baseline /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed
/opt/openroad/openroad_flow_scripts/flow/util/preprocessLib.py -i /opt/openroad/openroad_flow_scripts/flow/platforms/nangate45/lib/NangateOpenCellLibrary_typical.lib -o /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/baseline/HLS_output/Synthesis/bash_flow/openroad/objects/nangate45/main_kernel/base/lib/NangateOpenCellLibrary_typical.lib
Opening file for replace: /opt/openroad/openroad_flow_scripts/flow/platforms/nangate45/lib/NangateOpenCellLibrary_typical.lib
Commented 0 lines containing "original_pin"
Replaced malformed functions 0
Replaced capacitive_load_unit 0
Writing replaced file: /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/baseline/HLS_output/Synthesis/bash_flow/openroad/objects/nangate45/main_kernel/base/lib/NangateOpenCellLibrary_typical

### Optimized kernel

In [22]:
! scripts/run-openroad.sh optimized

# Approx. 42min to execute

/workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/optimized /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed
/opt/openroad/openroad_flow_scripts/flow/util/preprocessLib.py -i /opt/openroad/openroad_flow_scripts/flow/platforms/nangate45/lib/NangateOpenCellLibrary_typical.lib -o /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/optimized/HLS_output/Synthesis/bash_flow/openroad/objects/nangate45/main_kernel/base/lib/NangateOpenCellLibrary_typical.lib
Opening file for replace: /opt/openroad/openroad_flow_scripts/flow/platforms/nangate45/lib/NangateOpenCellLibrary_typical.lib
Commented 0 lines containing "original_pin"
Replaced malformed functions 0
Replaced capacitive_load_unit 0
Writing replaced file: /workspaces/soda/soda-opt/docs/tutorials/tensorflow/docker-version-executed/output/optimized/HLS_output/Synthesis/bash_flow/openroad/objects/nangate45/main_kernel/base/lib/NangateOpenCellLibrary_typi

## Comparison of synthesis results

* Display area
* Display power
* Calculate and display FLOPS/W

In [23]:
log_path_suffix='HLS_output/Synthesis/bash_flow/openroad/logs/nangate45/main_kernel/base/6_report.log'
rpt_path_suffix='HLS_output/Synthesis/bash_flow/openroad/reports/nangate45/main_kernel/base/6_finish.rpt'
gds_path_suffix='HLS_output/Synthesis/bash_flow/openroad/results/nangate45/main_kernel/base/6_final.gds'

### Baseline

In [24]:
baseline_log='output/baseline/'+log_path_suffix
baseline_rpt='output/baseline/'+rpt_path_suffix

In [25]:
power_multiplier = 1 # Open road reports power in W
total_power = ()

log_file=baseline_rpt

for l in open(log_file, 'r').readlines():
  if ("Total" in l and "Group" not in l):
    total_power=float(l.split()[4])*power_multiplier

log_file=baseline_log

for l in open(log_file, 'r').readlines():
  if ("Design area" in l):
    available_area=float(l.split()[2])
    utilization_area=float(l.split()[4].strip('%'))
  

print('Baseline accelerator:')
print('  total power consumption: {}W'.format(total_power))
print('  available chip area: {} um^2'.format(available_area))
print('  utilized chip area: {}%'.format(utilization_area))


baseline_total_power=total_power
baseline_available_area=available_area
baseline_utilization_area=utilization_area


Baseline accelerator:
  total power consumption: 0.0986W
  available chip area: 31013.0 um^2
  utilized chip area: 12.0%


### Optimized for runtime

In [26]:
optimized_log='output/optimized/'+log_path_suffix
optimized_rpt='output/optimized/'+rpt_path_suffix

In [27]:
power_multiplier = 1 # Open road reports power in W
total_power = ()

log_file=optimized_rpt

for l in open(log_file, 'r').readlines():
  if ("Total" in l and "Group" not in l):
    total_power=float(l.split()[4])*power_multiplier

log_file=optimized_log

for l in open(log_file, 'r').readlines():
  if ("Design area" in l):
    available_area=float(l.split()[2])
    utilization_area=float(l.split()[4].strip('%'))
  

print('Optimized accelerator:')
print('  total power consumption: {}W'.format(total_power))
print('  available chip area: {} um^2'.format(available_area))
print('  utilized chip area: {}%'.format(utilization_area))


optimized_total_power=total_power
optimized_available_area=available_area
optimized_utilization_area=utilization_area


Optimized accelerator:
  total power consumption: 0.114W
  available chip area: 285856.0 um^2
  utilized chip area: 12.0%


## Post place and route comparison

Considering a matrix multiply kernel has approximatelly 2xNxMxK arithmetic operations

And our selected kernel has the following sizes: 

```mlir
linalg.batch_matmul ins(%A, %B : memref<1x4x8xf32>, memref<1x8x4xf32>) 
                    outs(%C : memref<1x4x4xf32>)

```
M=4, K=8, N=4

We have approximatelly **256** floating point aritihmetic operations

In [28]:
giga_multiplier=1e9
flop_count = 256 # arithmetic float point operations
target_frequency = 200e+6 # 200MHz

optimized_runtime_in_s = optimized_runtime/target_frequency
baseline_runtime_in_s = baseline_runtime/target_frequency 

baseline_flops_per_watt= flop_count/baseline_runtime_in_s/baseline_total_power
optimized_flops_per_watt= flop_count/optimized_runtime_in_s/optimized_total_power


print("Execution in cycles of Baseline kernel:  {}".format(baseline_runtime))
print("Execution in cycles of Optimized kernel:   {}".format(optimized_runtime))

print("Speedup: \t\t\t{:.2f}x".format(baseline_runtime/optimized_runtime))
print("Area utilization overhead: \t {:.2f}x".format(optimized_utilization_area/baseline_utilization_area))
print("Area overhead: \t\t\t {:.2f}x".format(optimized_available_area/baseline_available_area))
print("Power overhead: \t\t {:.2f}x".format(optimized_total_power/baseline_total_power))

print("Baseline  \t\t\t {:.2f} GFLOPS/W ".format(baseline_flops_per_watt/giga_multiplier))
print("Optimized \t\t\t{:.2f} GFLOPS/W".format(optimized_flops_per_watt/giga_multiplier))


Execution in cycles of Baseline kernel:  1286
Execution in cycles of Optimized kernel:   56
Speedup: 			22.96x
Area utilization overhead: 	 1.00x
Area overhead: 			 9.22x
Power overhead: 		 1.16x
Baseline  			 0.40 GFLOPS/W 
Optimized 			8.02 GFLOPS/W


## Generated GDSII files

Output files can be found here:

* output/baseline/HLS_output/Synthesis/bash_flow/openroad/results/nangate45/main_kernel/base/6_final.gds
* output/optimized/HLS_output/Synthesis/bash_flow/openroad/results/nangate45/main_kernel/base/6_final.gds

### Baseline and Optimized Side by Side

![Side-By-Size](imgs/gds-side-by-side.png)

# Thank you!